<a href="https://colab.research.google.com/github/Chidiebere-Ogbuchi/Stage-C-Hamoye/blob/main/Stage_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")

In [18]:
# Reading the data into pandas dataframe
data = pd.read_csv("Telco-Customer-Churn.csv")
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [19]:
n_rows, n_columns = data.shape
print(f"Number of columns: {n_columns} columns\nNumber of rws: {n_rows} rows")

Number of columns: 21 columns
Number of rws: 7043 rows


In [20]:
data.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

### Preprocessing


* The numerical features should be scaled using StandardScaler, convert the output back to a dataframe and put back the column names.
* The categorical features are one-hot encoded using OneHotEncoder(set sparse_output to false), convert the output back to a dataframe and put back the column names.<br>
* Combine scaled numerical and one-hot encoded categorical features into train and test set dataframes (use pd.concat)<br>


* Perform initial data preparation by converting the 'TotalCharges' column to numeric values and filling missing values with 0.
* Convert the 'Churn' column to binary values, where 'No' is mapped to 0 and 'Yes' is mapped to 1.<br>
* Split the data into an 80-20 train-test split with a random state of “1”.<br>
* Select these features:  
   * categorical = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService','OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies','Contract', 'PaperlessBilling', 'PaymentMethod']
   * numerical = ['tenure', 'MonthlyCharges', 'TotalCharges']<br>

### Feature engineering


* The numerical features should be scaled using StandardScaler, convert the output back to a dataframe and put back the column names.
* The categorical features are one-hot encoded using OneHotEncoder(set sparse_output to false), convert the output back to a dataframe and put back the column names.<br>
* Combine scaled numerical and one-hot encoded categorical features into train and test set dataframes (use pd.concat)<br>

### Model Selection
* Use scikit learn to train a random forest and extra trees classifier, and use xgboost and lightgbm to train an extreme boosting model and a light gradient boosting model. Use random_state = 1 for training all models and evaluate on the test set.

### MODEL CREATION

In [21]:
# Convert 'TotalCharges' to numeric values
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

# Convert 'Churn' column to binary values
data['Churn'] = data['Churn'].map({'No': 0, 'Yes': 1})

# Split the data into an 80-20 train-test split with a random state of 1
X = data.drop('Churn', axis=1)
y = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Select features
categorical = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
               'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
               'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

numerical = ['tenure', 'MonthlyCharges', 'TotalCharges']

# Create a preprocessor for numerical features
numerical_preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Create a preprocessor for categorical features
categorical_preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse=False, drop='first'))
])

# Create a column transformer to apply the preprocessors
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_preprocessor, numerical),
        ('cat', categorical_preprocessor, categorical)
    ])

# Create a final pipeline including preprocessing and the classifiers
rf_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=1))
])

et_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', ExtraTreesClassifier(random_state=1))
])

# Fit the models
rf_model.fit(X_train, y_train)
et_model.fit(X_train, y_train)

# Preprocess data for XGBoost and LightGBM
preprocessor.fit(X_train)  # Fit the preprocessor on the entire training dataset
X_train_preprocessed = preprocessor.transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Convert the preprocessed data to int type for XGBoost
X_train_preprocessed = X_train_preprocessed.astype(int)
X_test_preprocessed = X_test_preprocessed.astype(int)

xgb_model = XGBClassifier(random_state=1)
lgbm_model = LGBMClassifier(random_state=1)

# Fit XGBoost and LightGBM models
xgb_model.fit(X_train_preprocessed, y_train)
lgbm_model.fit(X_train_preprocessed, y_train)

# Make predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test_preprocessed)
lgbm_predictions = lgbm_model.predict(X_test_preprocessed)

# Evaluate the models on the test set
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)
xgb_accuracy = accuracy_score(y_test, xgb_predictions)
lgbm_accuracy = accuracy_score(y_test, lgbm_predictions)

# Print the accuracy of Random Forest, Extra Trees, XGBoost, and LightGBM
print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)
print("XGBoost Accuracy:", xgb_accuracy)
print("LightGBM Accuracy:", lgbm_accuracy)


[LightGBM] [Info] Number of positive: 1521, number of negative: 4113
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000758 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 63
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.269968 -> initscore=-0.994785
[LightGBM] [Info] Start training from score -0.994785
Random Forest Accuracy: 0.8019872249822569
Extra Trees Accuracy: 0.78708303761533
XGBoost Accuracy: 0.7906316536550745
LightGBM Accuracy: 0.8041163946061036


#### Advancing ExtraTreesClassifier Model

In [13]:
# Define the hyperparameter grid
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]

hyperparameter_grid = {
    'classifier__n_estimators': n_estimators,
    'classifier__min_samples_split': min_samples_split,
    'classifier__min_samples_leaf': min_samples_leaf,
    'classifier__max_features': max_features
}

# Create the Extra Trees Classifier with the existing pipeline
et_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', ExtraTreesClassifier(random_state=1))
])

# Create the RandomizedSearchCV
randomized_search = RandomizedSearchCV(
    et_model,
    hyperparameter_grid,
    n_iter=10,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=1,
    random_state=1
)

# Fit the RandomizedSearchCV to find the best hyperparameters
randomized_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", randomized_search.best_params_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Hyperparameters: {'classifier__n_estimators': 50, 'classifier__min_samples_split': 5, 'classifier__min_samples_leaf': 8, 'classifier__max_features': None}
